# Python 代码中的空值

聚焦于 pandas 和 numpy 中的空值处理及识别

## 标准空值

Python 的 pandas 或 numpy 中空值有：

1. np.nan、np.NAN、np.NaN
2. pd.NA、pd.NaT
3. None（Python 内置的空值，类型为 NoneType）

其中，np.nan、np.NaN、np.NAN 基本等同，或者说 np.NaN、np.NAN 为 np.nan 的别名

使用 pd.dropna() 能去除所有标准的空值

In [23]:
import pandas as pd
import numpy as np
# 创建包含标准空值的DataFrame
df_standard = pd.DataFrame({
    'A': [1, np.nan, 3, None],           # np.nan 和 None
    'B': [pd.NA, 2, 3, 4],              # pd.NA
    'C': [pd.Timestamp('2023-01-01'), pd.NaT, pd.Timestamp('2023-01-03'), pd.Timestamp('2023-01-04')],  # pd.NaT
    'D': [np.NAN, 1, 2, 3],
    'E': [np.NaN, 4, 5, 6]
})
print("原始数据：")
print(df_standard)
print("\n检查空值：")
print(df_standard.isnull())
print("\n使用dropna()删除空值所在行后：")
print(df_standard.dropna())

原始数据：
     A     B          C    D    E
0  1.0  <NA> 2023-01-01  NaN  NaN
1  NaN     2        NaT  1.0  4.0
2  3.0     3 2023-01-03  2.0  5.0
3  NaN     4 2023-01-04  3.0  6.0

检查空值：
       A      B      C      D      E
0  False   True  False   True   True
1   True  False   True  False  False
2  False  False  False  False  False
3   True  False  False  False  False

使用dropna()删除空值所在行后：
     A  B          C    D    E
2  3.0  3 2023-01-03  2.0  5.0


## 伪装空值

如果是一些空字符，无法使用 pd.dropna() 去除。正确处理方式为：首先替换这些空字符，然后使用 pd.dropna()

In [9]:
# 创建包含伪装空值的DataFrame
df_fake_null = pd.DataFrame({
    'A': [1, '', 3, 4],                    # 空字符串
    'B': ['NULL', 2, 3, 4],               # 字符串'NULL'
    'C': ['NaN', 2, 3, 4],                # 字符串'NaN'
    'D': ['-', 2, 3, 4],                  # 连字符
    'E': ['n/a', 2, 3, 4]                 # 字符串'n/a'
})
print("包含伪装空值的数据：")
print(df_fake_null)
print("\n检查空值（注意：都是False）：")
print(df_fake_null.isnull())
print("\n使用dropna()后（没有任何行被删除）：")
print(df_fake_null.dropna())

包含伪装空值的数据：
   A     B    C  D    E
0  1  NULL  NaN  -  n/a
1        2    2  2    2
2  3     3    3  3    3
3  4     4    4  4    4

检查空值（注意：都是False）：
       A      B      C      D      E
0  False  False  False  False  False
1  False  False  False  False  False
2  False  False  False  False  False
3  False  False  False  False  False

使用dropna()后（没有任何行被删除）：
   A     B    C  D    E
0  1  NULL  NaN  -  n/a
1        2    2  2    2
2  3     3    3  3    3
3  4     4    4  4    4


In [10]:
# 先将伪装的空值替换为标准空值
df_cleaned = df_fake_null.replace(['', 'NULL', 'n/a', '-', 'NaN'], pd.NA)
print("\n替换伪装空值为pd.NA后：")
print(df_cleaned)
print("\n检查空值：")
print(df_cleaned.isnull())
print("\n现在可以用dropna()正确删除：")
print(df_cleaned.dropna())


替换伪装空值为pd.NA后：
      A     B     C     D     E
0     1  <NA>  <NA>  <NA>  <NA>
1  <NA>     2     2     2     2
2     3     3     3     3     3
3     4     4     4     4     4

检查空值：
       A      B      C      D      E
0  False   True   True   True   True
1   True  False  False  False  False
2  False  False  False  False  False
3  False  False  False  False  False

现在可以用dropna()正确删除：
   A  B  C  D  E
2  3  3  3  3  3
3  4  4  4  4  4


## 识别空值

1. 如果某一列期望是数值型（int, float）或日期型，但 df.info() 显示其类型是 object (通常代表字符串)，这通常意味着该列混杂了非数值/日期字符，其中就可能包含伪装的空值

2. info() 会统计非空数，可以识别标准空值

3. value_counts() 会列出该列中所有唯一值及其出现的次数。伪装的空值字符串（如 "", "NA", "N/A", "null", "-", "?" 等）会和它们的频次一起显示出来

In [ ]:
df_standard.info()
df_standard['A'].value_counts()

df_fake_null.info()
df_fake_null['A'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       4 non-null      object
 1   B       4 non-null      object
 2   C       4 non-null      object
 3   D       4 non-null      object
 4   E       4 non-null      object
dtypes: object(5)
memory usage: 288.0+ bytes


## 补充

### 关于 np.nan、np.NaN、np.NAN 的关系

参考：https://stackoverflow.com/questions/53436339/difference-between-np-nan-and-np-nan

In [ ]:
# 不能判断 nan 是否相等（equality）
print(np.nan == np.NaN)
print(np.nan == np.NAN)
print(np.NaN == np.NAN)

print(type(np.nan))
print(type(np.NaN))
print(type(np.NAN))

False
False
False
<class 'pandas._libs.missing.NAType'>
<class 'float'>
<class 'float'>
